#  Байесовский подход к оценке А/Б тестов

*При проведении А/Б-тестов в мобильных приложениях и веб-сервисах требуется ответить на вопросы "Какой вариант лучше и насколько?", "Каковы оценки целевой метрики в каждом варианте?", "Насколько уверены в оценке?", "Сколько должен продолжаться эксперимент?".
На примере двух случайных процессов с бинарными исходами показано, как в рамках байесовского подхода ответить на вопросы выше и выбрать процесс с большей вероятностью успеха.*

**Содержание:**
* [Введение](#Введение)  
* [Общая идея байесовского подхода](#Общая-идея-байесовского-подхода)
* [Оценка вероятности в эксперименте с двумя исходами](#Оценка-вероятности-в-эксперименте-с-двумя-исходами)
* &nbsp; &nbsp; [Аналитический расчет апостериорного распределения](#Аналитический-расчет-апостериорного-распределения)
* [Сравнение конверсий в А/Б тесте](#Сравнение-конверсий-в-А/Б-тесте)
* &nbsp; &nbsp; [Оценка величины параметра в группах и область наиболее вероятных значений](#Оценка-величины-параметра-в-группах-и-область-наиболее-вероятных-значений)
* &nbsp; &nbsp; [Какая группа лучше и насколько?](#Какая-группа-лучше-и-насколько?)
* &nbsp; &nbsp; [Зависимость точности оценки параметров от размера выборки](#Зависимость-точности-оценки-параметров-от-размера-выборки)
* &nbsp; &nbsp; [Сколько должен продолжаться эксперимент?](#Сколько-должен-продолжаться-эксперимент?)
* [Дополнение: динамика по дням](#Дополнение:-динамика-по-дням)
* [Приложение: сопряженное априорное распределение к биномиальному](#Приложение:-сопряженное-априорное-распределение-к-биномиальному)
* [Заключение](#Заключение)  

## Введение

При развитии мобильных приложений и веб-сервисов часто возникает идея внести в них те или иные изменения. Ожидается, что это улучшит ключевые метрики сервиса - финансовые, аудиторные, конверсии, вовлеченность и др. Возможный пример - изменение цен или тарифной линейки. Хотя при повышении цен конверсия в покупку скорее всего снизится, итоговая прибыль может вырасти.  

<center>
<img src="../figs/experiment_versions.png" alt="experiment_versions" width="400"/>
</center>

Точный эффект от изменений непредсказуем. Более того, может оказаться, что функциональность ухудшает продукт. По оценкам, только около трети реализованных изменений приводят к положительным результатам [[MicroExp](https://www.microsoft.com/en-us/research/publication/online-experimentation-at-microsoft/)]. Поэтому необходимо измерять эффект от новой функциональности.

После запуска эффект не всегда может быть виден сразу (см. график ниже). Если функциональность не вызвала резких изменений, ее влияние может быть незаметно на фоне случайных колебаний метрик. Кроме того, могут произойти изменения в других частях продукта, привлекаемом трафике или общей активности аудитории, которые также повлияют на целевую метрику. Например, запуск рекламной акции. Поэтому изменения метрик после релиза не всегда можно объяснять именно новой функциональностью.

На графике ниже показаны варианты влияния изменений на некую условную конверсию.  
Original - исходный вариант, Original + 10% - улучшение на 10%, но на глаз эффект не ясен, 
Original - 50% - заметное ухудшение.  

<center>
<img src="../figs/feature_effects.png" alt="feature_effects"/>
</center>

Одним из способов оценки новой функциональности является проведение А/Б-теста.
В этом подходе запускают версию без изменений и измененные варианты сервиса параллельно, распределяют пользователей между этими вариантами и сравнивают интересующие метрики. Параллельный запуск версий позволяет снизить различия из-за факторов, не связанных с тестируемыми изменениями.

В проведении АБ-тестов есть много нюансов [[TrustworthyAB]((https://www.amazon.com/gp/product/B0845Y3DJV)].
Но общая схема примерно следующая (см. рисунок). При попадании на сайт или в приложение пользователь случайным образом определяется в одну из экспериментальных групп. В каждой группе собираются данные и вычисляются интересующие метрики.
Полученные значения сравниваются между собой. Эксперимент прекращается, если становится понятно, что одна из групп лидирует (иногда - при прохождении определенного количества пользователей, по истечении определенного времени или нецелесообразности дальнейшего проведения). Принимается решение о дальнейших действиях - как правило, о выборе одного из вариантов для всех пользователей.

![ab_tests](../figs/ab_test.png)

В результате эксперимента нужно получить ответ на несколько вопросов, чтобы определиться с дальнейшими решениями по тестируемой функциональности. 

В конечном счете нужно сказать, какая группа лучше и насколько.       
Для этого понадобится посчитать оценки целевой метрики в каждой группе.  

Значения метрик нужны для сравнения с ценой поддержки.  
Также может понадобиться оценить различие между вариантами, которое накопится, например, за год.

При сравнении вариантов остается неопределенность.
Поэтому добавляется вопрос "насколько уверены в оценке"?

Как правило по мере увеличения количества данных - т.е. по мере роста количества пользователей, принявших участие в эксперименте - неопределенность в оценках снижается. Поэтому еще один вопрос - сколько должен продолжаться эксперимент, чтобы можно было выбрать один из вариантов с определенной степенью уверенности?

<div style="font-size: 20px">  

Основные вопросы при А/Б-тестировании:

<ul>
<li>Каковы оценки целевой метрики в каждом варианте?</li> 
<li>Какой вариант лучше? Насколько лучше?</li>
<li>Насколько уверены в оценке?</li>
<li>Сколько должен продолжаться эксперимент?</li>
</ul>

</div>

Например, в имеющейся выборке среднее значение метрики в группе A равно a_mean, а в группе B - b_mean. При выборе варианта A для всех пользователей среднее значение метрики с вероятностью 90% будет находится в диапазоне (a_min, a_max), при выборе варианта B - с вероятностью 90% в диапазоне (b_min, b_max). По имеющимся данным группа B окажется лучше группы A с вероятностью 70%. Ожидаемое значение B/A = 1.05. Чтобы говорить о p(B) > p(A) с уверенностью 90% нужно еще около N пользователей, что займет d дней.   

Вероятность здесь и далее понимается в субъективном смысле - как мера уверенности в том или ином исходе процесса с несколькими возможными исходами [[SubjProb](https://en.wikipedia.org/wiki/Probability_interpretations#Subjectivism), [UU](https://www.amazon.co.uk/Understanding-Uncertainty-Wiley-Probability-Statistics-ebook/dp/B00GYVM33Q)].

Существуют различные подходы к оценке А/Б-тестов. Один из наиболее популярных - аппарат проверки статистических гипотез. 
Общая идея - предположить, что разницы между вариантами нет и проверить, насколько такое предположение объясняет фактически полученные данные.
Обсуждение особенностей этого и других методов - отдельная тема. 
Ниже речь пойдет только о байесовском подходе.  

Содержание следующее:
- кратко обсуждены основные идеи байесовского подхода
- на примере случайного процесса с двумя возможными исходами проводится построение модели процесса и оценка параметров модели
- на примере двух случайных процессов обсуждается сравнение моделей и способ ответа на вопросы для А/Б-теста

В дополнениях обсуждаются вспомогательные вопросы. 

# Общая идея байесовского подхода

В байесовском подходе [[SGBS](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [SR](https://www.amazon.co.uk/Statistical-Rethinking-Bayesian-Examples-Chapman/dp/036713991X/ref=sr_1_1)] строят математическую модель, объясняющую фактические данные.   
Далее проводится сравнение моделей.  
Это более общая задача, чем сравнение распределений.

Данные можно объяснить несколькими различными моделями.  
Вместо выбора только одной модели используют группу моделей, но для каждой модели считают "вес" относительно других.  
Роль "веса" играет вероятность $P(model|data)$ - уверенность в модели $model$ при условии, что наблюдаются фактические данные $data$. 

Для расчета $P(model|data)$ используется связь с $P(data|model)$ - вероятностью наблюдения данных $data$ в рамках выбранной модели $model$. Связь выражается соотношением Байеса:

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} .
$$

Используется следующая терминология:  
$P(model | data)$ - апостериорное распределение вероятности,  
$P(data | model)$ - функция правдоподобия,  
$P(model)$ - априорное распределение вероятности,  
$P(data)$ не имеет специального названия.  

Обычно форму модели фиксируют. Меняют только параметры.  

Последовательность действий при использовании байесовского подхода следующая (см. также рис. ниже).  
Выбирается набор возможных моделей $model$ (или форма модели и область параметров для нее).  
Для каждого набора параметров задается априорная вероятность $P(model)$.  
Вычисляется функция правдоподобия $P(data|model)$ - вероятность получить данные в рамках выбранной модели.    
Вычисляется апостериорная вероятность $P(model|data)$.  
Анализируются свойства моделей и постериорных распределений.  

<center>
<img src="../figs/bayes_scheme.png" alt="bayes_scheme"/>
</center>

Важным этапом является проверка модели.  
Можно промахнуться с формой модели (пытаться аппроксимировать синусоиду прямой линией).  
Кажется, в общем случае решения нет.  
В "простых" случаях есть теорема полноты (теорема Де-Финетти).  
На практике модель собирают из изученного набора аналитических распределений.

## Оценка вероятности в эксперименте с двумя исходами

В качестве примера байесовского моделирования можно рассмотреть оценку 
вероятностей в эксперименте с двумя исходами.

Такой эксперимент может служить моделью конверсии пользователей в целевое действие.
Каждый пользователь сервиса может выполнить или не выполнить целевое действие.
Предполагается, что вероятность одинакова и не зависит от других пользователей.

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Проводится серия $N$ экспериментов с 2 возможными исходами (схема Бернулли [[BernoulliProcess](https://en.wikipedia.org/wiki/Bernoulli_process)]).  
Условно каждый эксперимент можно называть "броском монеты".  
Пусть вероятность одного исхода в каждом "броске" $p = 0.7$, второго $(1-p) = 0.3$ .

In [ ]:
outcomes = ['heads', 'tails']
n_trials = 1000
p_heads = 0.7
probs = [p_heads, 1 - p_heads]

results = np.random.choice(outcomes, size=n_trials, p=probs)
print(results[:7], '...')
print()

n_heads = np.sum(results == 'heads')
print(f'n_heads: {n_heads}, n_tails: {n_trials-n_heads}')
print(f'exact p_heads: {p_heads}')
print(f'avg p_heads: {n_heads / n_trials}')

В n_trials экспериментах получилось n_heads выпадений орла.  

Предположим, что точное значение $p$ неизвестно и его нужно оценить по $n_{heads}$ и $n_{trials}$.  
Для общего числа бросков $n_{trials}$ количество успехов $n_{heads}$ могло получиться при различных значениях параметра $p$.  
Один из способов оценить вероятные значения $p$ - перебор возможных значений и расчет вероятности для каждого.  

Для упрощения вместо непрерывного набора значений параметра $p$ на интервале от 0 до 1 можно ограничиться перебором дискретного набора возможных значений на равномерной сетке. Пусть в сетке M узлов, тогда

$$
p \in \left\{ 0, \frac{1}{M-1}, \frac{2}{M-1}, \dots, 1 \right\} .
$$

С помощью байесовского метода можно оценить вероятности различных значений параметра $p$, т.е. $P(p | data)$:  

$$
P(p | data) = \frac{ P(data | p) P(p) }{P(data)} .
$$

Нужно задать функцию правдоподобия $P(data | p)$ и априорное распределение вероятности $P(p)$ параметра $p$. Знаменатель $P(data)$ играет роль нормировки и при заданных $P(data | p)$ и $P(p)$ может быть вычислен как $P(data) = \sum_p P(data|p) P(p)$.

В качестве функции правдоподобия $P(data | p)$ можно выбрать биномиальное распределение [[BinomDist](https://en.wikipedia.org/wiki/Binomial_distribution), [SciPyBinom](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html)].  
Оно моделирует вероятность получить $k$ "успехов" в серии из $N$ экспериментов с двумя возможными исходами при вероятности "успеха" $p$. В данном случае $k=n_{heads}, N=N_{trials}$.

$$
P(data| p) = \mbox{Binomial}(p ; k=n_{heads}, N=N_{trials}),
\\ 
\mbox{Binomial}(p ; k, N)= {N \choose k} p^{k} (1-p)^{N-k} .
$$

Все $p$ будем считать равновероятными. Всего перебирается $M$ значений $p$, поэтому

$$
P(p) = \frac{1}{M} .
$$

В итоге:

$$
p \in \left\{ 0, \frac{1}{M-1}, \frac{2}{M-1}, \dots, 1 \right\} ,
\\
P(p) = \frac{1}{M} ,
\\
P(data | p) = \mbox{Binomial}(p ; n_{heads}, N_{trials}) ,
\\
\mbox{Binomial}(p ; k, N)= {N \choose k} p^{k} (1-p)^{N-k} ,
\\
P(data) = \sum_p P(data | p) P(p) .
$$

In [ ]:
grid_points = 1001
p_grid = np.linspace(0, 1, grid_points)
p_grid_prior = [ 1.0 / grid_points for x in p_grid]
#print(p_grid)
#print(p_grid_prior)

likelihood = [stats.binom.pmf(n_heads, n_trials, p) for p in p_grid]
p_posterior = [l * pr for (l, pr) in zip(likelihood, p_grid_prior)]
p_posterior = p_posterior / sum(p_posterior)
#print(likelihood)
#print(p_posterior)

fig = go.Figure(data=go.Scatter(x=p_grid, y=p_posterior, mode='lines+markers'))
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob',
                  hovermode="x")
fig.show()

print('exact p_heads:', p_heads)
print('avg p_heads:', n_heads / n_trials)
print('max p_heads estimate:', p_grid[np.argmax(p_posterior)])

Таким образом можно получить оценку вероятностей различных значений параметра $p$.

По-хорошему, выбранную модель нужно проверить.  
Проверка моделей пока не обсуждается.

## Аналитический расчет апостериорного распределения

В отдельных случаях апостериорное распределение вероятности удается посчитать аналитически. Это удается сделать не всегда, но для определенных комбинаций функции правдоподобия и априорного распределения удается. Выражение для апостериорного распределения:

$$
P(p | data) = \frac{ P(data | p) P(p) }{P(data)}.
$$

Пусть функция правдоподобия задается биномиальным распределением [[BinomDist](https://en.wikipedia.org/wiki/Binomial_distribution)]:

$$
P(data | p) = Binom(p, s, N) = {N \choose s} p^s (1-p)^{N-s} .
$$

Значение параметра $p$ непрерывно, а априорная плотность вероятности равномерна:  

$$
p \in [0, 1] ,
\\
P(p) = Uniform(0,1) = \frac{1}{1 - 0} = 1 .
$$

При непрерывном распределении параметров $P(data)$ может быть представлен в виде интеграла по всей области значений параметров:

$$
P(data) = \int \limits_0^1 P(data | p) P(p) dp .
$$

При подстановке в соотношение Байеса

$$
P(p | data) = \frac{ p^s (1-p)^{N-s} }{\int_0^1 p^s (1-p)^{N-s} dp} .
$$

Это выражение по форме совпадает с т.н. бета-распределением [[BetaDist](https://en.wikipedia.org/wiki/Beta_distribution), [SciPyBeta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html)]

$$
Beta(x; \alpha, \beta) 
= 
\frac{x^{\alpha-1}(1-x)^{\beta-1}}{\int_0^1 u^{\alpha-1} (1-u)^{\beta-1} du}
=
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)}
x^{\alpha-1}(1-x)^{\beta-1} . 
$$

Таким образом апостериорное распределение задается бета-распределением с параметрами $\alpha = s + 1$, $\beta = N - s + 1$:

$$
P(p | data) = Beta(p; \alpha, \beta) ,
\\
\alpha = s + 1, \quad \beta = N - s + 1 .
$$

Вид бета-распределения при различных значениях параметров обсуждается в [приложении](#Приложение:-сопряженное-априорное-распределение-к-биномиальному).

Можно сравнить аналитическую плотность распределения с посчитанной выше численной. Для этого нужно посчитать вероятность тех же значений параметра $p$, что и в сетке прошлой модели. Значения плотности вероятности в конкретной точке $p$ будут отличаться от дискретной вероятности выше. Формально для сравнения нужно приблизить непрерывную аналитическую плотность вероятности дискретным распределением вероятности. Практически при аналогичной нормировке значения совпадают.  

In [ ]:
def posterior_for_binom_and_uniform_prior(p, n_heads, n_trials):
    alpha_prior = 1
    beta_prior = 1
    alpha_post = alpha_prior + n_heads
    beta_post = beta_prior + (n_trials - n_heads)
    return stats.beta.pdf(p, alpha_post, beta_post)

In [ ]:
grid_points = 1001
p_grid = np.linspace(0, 1, grid_points)

p_an_posterior = [posterior_for_binom_and_uniform_prior(p, n_heads, n_trials) for p in p_grid]
p_an_posterior = p_an_posterior / np.sum(p_an_posterior)

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_grid, y=p_posterior, mode='lines+markers', name='num'))
fig.add_trace(go.Scatter(x=p_grid, y=p_an_posterior, mode='lines', name='an'))
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob',
                  hovermode="x")

fig.show()

Можно показать, что если функция правдоподобия задана биномиальным распределением, априорная вероятность - бета-распределением, то апостериорная вероятность также будет выражаться бета-распределением, но с другими значениями параметров:

$$
\begin{aligned}
P(data | p) = Binom(p, s, N)
\\
P(p) = Beta(p; \alpha, \beta)
\end{aligned}
\longrightarrow
\begin{aligned}
& P(p | data) = Beta(p; \alpha', \beta'),
\\
& \alpha' = \alpha + s, \; \beta' = \beta + (N-s).
\end{aligned}
$$

Говорят, что бета-распределение является сопряженным априорным распределением к биномиальному [[ConjPrior](https://en.wikipedia.org/wiki/Conjugate_prior)]. Это можно использовать для упрощения вычислений и задания неравномерных априорных распределений - см. [приложение](#Приложение:-сопряженное-априорное-распределение-к-биномиальному).

# Сравнение конверсий в А/Б тесте

Как модель АБ-теста можно использовать 2 схемы Бернулли с вероятностями $p_A$ и $p_B$. 

In [ ]:
exp_info = pd.DataFrame([
    {'group' :'A', 'p_exact': 0.03, 'N': 3000},
    {'group': 'B', 'p_exact': 0.035, 'N': 3000}],
    columns=['group', 'p_exact', 'N']
).set_index('group')
exp_info

In [ ]:
ab_outcomes = ['buy', 'leave']

probs_a = [exp_info['p_exact']['A'], 1 - exp_info['p_exact']['A']]
probs_b = [exp_info['p_exact']['B'], 1 - exp_info['p_exact']['B']]

results_a = np.random.choice(ab_outcomes, size=exp_info['N']['A'], p=probs_a)
results_b = np.random.choice(ab_outcomes, size=exp_info['N']['B'], p=probs_b)

print('A: ', results_a[:7], '...')
print('B: ', results_b[:7], '...')
print()

exp_info['N_buy'] = [np.sum(results_a == 'buy'), np.sum(results_b == 'buy')]
exp_info['p_mean'] = exp_info['N_buy'] / exp_info['N']
exp_info

## Оценка величины параметра в группах и область наиболее вероятных значений

Байесовская оценка параметра в каждой группе:

In [ ]:
p_grid = np.linspace(start=0, stop=1, num=3001)

p_posterior_a = np.array([posterior_for_binom_and_uniform_prior(p, exp_info['N_buy']['A'], exp_info['N']['A']) for p in p_grid])
p_posterior_b = np.array([posterior_for_binom_and_uniform_prior(p, exp_info['N_buy']['B'], exp_info['N']['B']) for p in p_grid])

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_grid, y=p_posterior_a, mode='lines', name='A', line_color='red'))
fig.add_trace(go.Scatter(x=p_grid, y=p_posterior_b, mode='lines', name='B', line_color='blue'))
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x")
fig.update_layout(xaxis_range=[0, 0.1])
fig.show()

Чтобы охарактеризовать значение параметра одним числом, можно использовать средние значения $p_{mean}$ из выборки.  
Другой возможный вариант - использовать $p$ с максимальной апострериорной плотностью вероятности.  

In [ ]:
exp_info['max_p_estimate'] = [p_grid[np.argmax(p_posterior_a)], p_grid[np.argmax(p_posterior_b)]]
exp_info

Чтобы передать неопределенность, удобно использовать интервал значений.  
Интервал может включать, например, 90% распределения.  
Т.к. в распределении есть один пик, удобно чтобы центр интервала совпадал с областью с наибольшей плотности вероятности.  
Интервал с наибольшей плотностью вероятности далее обозначается HPDI - Highest Posterior Density Interval.

In [ ]:
def hpdi_for_binom_and_uniform_prior(hpdi, n_heads, n_trials):
    p_grid = np.linspace(start=0, stop=1, num=3001)
    p_posterior = np.array([posterior_for_binom_and_uniform_prior(p, n_heads, n_trials) for p in p_grid])
        
    norm = np.sum(p_posterior)
    n_start = np.argmax(p_posterior)
    n_left = n_start
    n_right = n_start
    s = p_posterior[n_start]

    while s < hpdi * norm:
        next_left = p_posterior[n_left - 1]
        next_right = p_posterior[n_right + 1]
        if next_left > next_right:
            n_left = n_left - 1
            s = s + next_left
        elif next_left < next_right:
            n_right = n_right + 1
            s = s + next_right
        else:
            n_left = n_left - 1
            n_right = n_right + 1
            s = s + next_left + next_right
    return(p_grid[n_left], p_grid[n_right])

In [ ]:
hpdi_left_a, hpdi_right_a = hpdi_for_binom_and_uniform_prior(0.9, exp_info['N_buy']['A'], exp_info['N']['A'])
hpdi_left_b, hpdi_right_b = hpdi_for_binom_and_uniform_prior(0.9, exp_info['N_buy']['B'], exp_info['N']['B'])

exp_info['90% HPDI'] = [(hpdi_left_a, hpdi_right_a), (hpdi_left_b, hpdi_right_b)]
exp_info

In [ ]:
left_a = exp_info['90% HPDI']['A'][0]
right_a = exp_info['90% HPDI']['A'][1]
left_b = exp_info['90% HPDI']['B'][0]
right_b = exp_info['90% HPDI']['B'][1]

fig = go.Figure()
fig.add_trace(go.Scatter(x=p_grid, y=p_posterior_a, mode='lines', name='A', line_color='red'))
fig.add_trace(go.Scatter(x=p_grid[(p_grid > left_a) & (p_grid < right_a)], 
                         y=p_posterior_a[(p_grid > left_a) & (p_grid < right_a)], 
                         mode='lines', fill='tozeroy', line_color='red',
                         name='90% HPDI A'))
fig.add_trace(go.Scatter(x=p_grid, y=p_posterior_b, mode='lines', name='B', line_color='blue'))
fig.add_trace(go.Scatter(x=p_grid[(p_grid > left_b) & (p_grid < right_b)], 
                         y=p_posterior_b[(p_grid > left_b) & (p_grid < right_b)], 
                         mode='lines', fill='tozeroy', line_color='blue',
                         name='90% HPDI B'))
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x")
fig.update_layout(xaxis_range=[0, 0.1])
fig.show()

print(f"""
    90% HPDI A: ({p_grid[p_grid > left_a][0]:.2%} - {p_grid[p_grid < right_a][-1]:.2%})
    90% HPDI B: ({p_grid[p_grid > left_b][0]:.2%} - {p_grid[p_grid < right_b][-1]:.2%})   
""")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[exp_info['p_mean']['A']],
                         y=['A'],
                         customdata=[f'90%-HPDI: ({hpdi_left_a:.3f}, {hpdi_right_a:.3f})'],
                         hovertemplate = 'Mean: %{x:.3f}<br> %{customdata}',
                         error_x=dict(
                             type='data',
                             symmetric=False,
                             arrayminus=[exp_info['p_mean']['A'] - p_grid[p_grid > hpdi_left_a][0]],
                             array=[p_grid[p_grid < hpdi_right_a][-1] - exp_info['p_mean']['A']],
                             visible=True),
                        name='A', line_color='red'))
fig.add_trace(go.Scatter(x=[exp_info['p_mean']['B']],
                         y=['B'],
                         customdata=[f'90%-HPDI: ({hpdi_left_b:.3f}, {hpdi_right_b:.3f})'],
                         hovertemplate = 'Mean: %{x:.3f}<br> %{customdata}',
                         error_x=dict(
                             type='data',
                             symmetric=False,
                             arrayminus=[exp_info['p_mean']['B'] - p_grid[p_grid > hpdi_left_b][0]],
                             array=[p_grid[p_grid < hpdi_right_b][-1] - exp_info['p_mean']['B']],
                             visible=True),
                         name='B', line_color='blue'))
#fig.update_layout(hovertemplate = 'Mean: %{x:$.2f}<br> 80-HPDI: %{customdata[0]}')
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="closest")
fig.update_layout(xaxis_range=[0, 0.1])
fig.show()

Значение с максимальной апострериорной вероятностью и HPDI не используются для точного принятия решений.  
Они нужны для быстрой характеристики распределения.
Это бывает удобно, например, для динамики по дням (см. далее).  
Выбор 90% в HPDI - условность.  
Можно использовать другие значения.  

## Какая группа лучше и насколько?

Чтобы понять, какой вариант лучше, нужно оценить вероятность $p_B > p_A$.  

Для оценки "насколько один вариант лучше другого" можно использовать разность $p_B - p_A$, отношение $p_B/p_A$ или относительную разность $(p_B-p_A)/p_A$. Для этого определяется соответствующая случайная величина, например $p_Z = p_B - p_A$ или $p_Z=p_B/p_A$, и вычисляется ее распределение. 

Вероятность $P(p_B > p_A)$ будет совпадать с вероятностями $P(p_B - p_A > 0)$ и $P(p_B/p_A > 1)$. 

Также эти распределения можно использовать для ответа на вопросы об ожидаемом значении $E(p_B/p_A)$ или, например, вероятности того, что величина эффекта больше определенного значения $p_B/p_A > 1.05$. 

Выражения для разности и отношения задаются свертками плотностей распределений [[ProbConv](https://en.wikipedia.org/wiki/Convolution_of_probability_distributions), [ProbRatio](https://en.wikipedia.org/wiki/Ratio_distribution)]. Вместо аналитического расчета распределения разности или отношения часто используют приближенный метод - сэмплируют апостериорные распределения и анализируют свойства полученных выборок.

Ниже построено распределение отношений $p_B / p_A $ в сэмплах.    
При расчете отношений требуется аккуратность если в знаменателе значимая плотность вероятности вблизи 0.   
Например, отношение нормальных распределений с центром в 0 не имеет определенного среднего и дисперсии (см. [[CauchyDist](https://en.wikipedia.org/wiki/Cauchy_distribution)]).  
Если вероятность 0 не очень большая, то значения можно проигнорировать, хотя это может исказить распределение.    
Поскольку в группе $A$ есть покупки, то точно $p_A=0$ выпадать не должно.  

In [ ]:
#from numpy.random import default_rng
#rng = default_rng(17)

def posterior_sample_for_binom_and_uniform_prior(ns, ntotal, n_sample):
    alpha_prior = 1
    beta_prior = 1
    a = alpha_prior + ns
    b = beta_prior + (ntotal - ns) 
    return np.random.beta(a, b, n_sample)

In [ ]:
n_sample = 100000

post_sample_a = posterior_sample_for_binom_and_uniform_prior(exp_info['N_buy']['A'], exp_info['N']['A'], n_sample)
post_sample_b = posterior_sample_for_binom_and_uniform_prior(exp_info['N_buy']['B'], exp_info['N']['B'], n_sample)

post_sample_rel = post_sample_b / post_sample_a

fig = go.Figure()
fig.add_trace(go.Histogram(x=post_sample_rel, histnorm='probability density', 
                           name='B/A', marker_color='red',
                           opacity=0.6))
fig.add_vline(x=1, line_dash="dash")

fig.update_layout(title='B/A',
                  xaxis_title='B/A',
                  yaxis_title='Prob Density',
                  barmode='overlay')
fig.show()

print(f"Expected(B/A) = {np.mean(post_sample_rel):.2f}")

Чтобы получить оценку вероятности $P(p_B/p_A > 1)$, нужно найти долю точек, попавшую в интересующий диапазон:

In [ ]:
pb_gt_pa = len(post_sample_rel[post_sample_rel > 1]) / len(post_sample_rel)
print(f'P(p_B/p_A > 1): {pb_gt_pa}')

Аналогичным способом можно оценить вероятность, что эффект больше определенной величины.  
Например, $P(p_B/p_A > 1.05)$:

In [ ]:
print(f'P(p_B/p_A > 1.05): {len(post_sample_rel[post_sample_rel > 1.05]) / len(post_sample_rel)}')

В итоге:

In [ ]:
pb_gt_pa = len(post_sample_rel[post_sample_rel > 1]) / len(post_sample_rel)
pa_gte_pb = len(post_sample_rel[post_sample_rel <= 1]) / len(post_sample_rel)

x = ['p_B <= p_A', 'p_B > p_A', ]
y = [pa_gte_pb, pb_gt_pa]
colors = ['red', 'green']

fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=y, marker_color=colors, width=0.3))
fig.update_layout(yaxis_range=[0,1])
fig.update_layout(
     autosize=False,
     width=800,
     height=500)
fig.show()

b_to_a_mean = np.mean(post_sample_rel)
exp_info['Prob Optimal Variant'] = [pa_gte_pb, pb_gt_pa]
exp_info['Expected Rel to A'] = [1, b_to_a_mean]
exp_info.T

### Зависимость точности оценки параметров от размера выборки

Есть ожидание, что точность оценки параметров будет зависеть от размера выборки.  
Чем больше данных - тем точнее оценка.

На графике ниже показаны оценки апостериорного распределения для различного количества наблюдений. Видно, что с ростом числа наблюдений ширина пиков уменьшается - т.е. уменьшается неопределенность в оценке параметра.

In [ ]:
outcomes = ['heads', 'tails']
p_heads = 0.7
probs = [p_heads, 1 - p_heads]

N = [30, 300, 500, 1000, 3000]
hpdi = 0.9
hpdis = []

fig = go.Figure()
grid_points = 1001
p_grid = np.linspace(0, 1, grid_points)

for n in N:
    res = np.random.choice(outcomes, size=n, p=probs)
    n_heads = np.sum(res == 'heads')
    p_posterior = np.array([posterior_for_binom_and_uniform_prior(p, n_heads, n) for p in p_grid])
    hpdi_left, hpdi_right = hpdi_for_binom_and_uniform_prior(hpdi, n_heads, n)
    hpdi_width = np.abs(hpdi_right - hpdi_left)
    hpdis = hpdis + [hpdi_width]
    fig.add_trace(go.Scatter(x=p_grid, y=p_posterior, mode='lines', name=f"n = {n}"))    
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x")
fig.show()

В конечном счете интересует, как меняется вероятность $P(p_B > p_A)$ с ростом числа наблюдений?   
Сколько нужно ждать, пока уверенность в одном из вариантов дойдет до приемлемого уровня?

Вначале можно оценить зависимость ширины апостериорных распределений от $N$.  
В рассматриваемой модели апостериорное распределение описывается бета-распределением.  
Ширину распределения можно охарактиризовать дисперсией, которая для бета-распределения вычисляется аналитически [[BetaDist](https://en.wikipedia.org/wiki/Beta_distribution)]

$$
P(p | data) = Beta(p; \alpha, \beta)
\\
\alpha = s + 1, \quad \beta = N - s + 1
\\
\sigma^2=\frac{\alpha \beta}{(\alpha+\beta)^2 (\alpha+\beta+1)}
$$

С ростом числа наблюдений $N$ дисперсия убывает как $1/N$, стандартное отклонение - как $1/\sqrt{N}$

$$
\lim_{n \to \infty} \sigma = \frac{1}{\sqrt{N}}
$$

Т.к. дисперсия бета-распределения убывает как $1/\sqrt{N}$, то можно ожидать, что ширина 90%-HPDI будет убывать примерно так же.  
Это подтверждается расчетами:

In [ ]:
fig = go.Figure()
for n, hpdi in zip(N, hpdis):
    fig.add_trace(go.Scatter(x=[n], y=[hpdi], mode='markers', name=f"n = {n}"))
x_fit = np.arange(N[0], N[-1])
c_fit = hpdis[-1] * np.sqrt(N[-1])
y_fit = [c_fit / np.sqrt(n) for n in x_fit]
fig.add_trace(go.Scatter(x=x_fit, y=y_fit, name='Fit C/sqrt(N)'))
fig.update_layout(title='90% HPDI Width',
                  xaxis_title='N',
                  yaxis_title='90% HPDI Width',
                  hovermode="x")
fig.show()

Поэтому чтобы "сузить" значение HPDI в 2 раза нужно в 4 раза больше наблюдений, чем на текущий момент.  
Например, если HPDI = (2.0 - 4.0) при N=3000, то для HPDI примерно (2.5-3.5) нужно N около 12000.

С ростом $N$ можно ожидать повышения уверенности в том, какая из групп лучше.  
Зависимость $P(p_B / p_A > 1)$ и $P(p_B > p_A)$ от $N$ аналитически оценить сложнее.  
Можно оценить численно.

In [ ]:
def prob_pb_gt_pa(p_a, p_b, N_a, N_b = None):
    N_b = N_a if N_b is None else N_b
    n_sample = 100000
    post_sample_a = posterior_sample_for_binom_and_uniform_prior(p_a * N_a, N_a, n_sample)
    post_sample_b = posterior_sample_for_binom_and_uniform_prior(p_b * N_b, N_b, n_sample)
    post_sample_diff = post_sample_b - post_sample_a
    prob_b_gt_a = len(post_sample_diff[post_sample_diff > 0]) / len(post_sample_diff)
    return prob_b_gt_a

Ниже приведена зависимость вероятности $P(p_B > p_A)$ от числа наблюдений для значений
$p_A$ равных 1% и 10%. $p_B$ выбирается равным $1.1 p_A$, либо $1.3 p_A$.  
C ростом числа наблюдений $P(p_B > p_A)$ в каждом случае стремится к единице.  
Однако при малых N скорость роста отличается и зависит как от относительной разности, так и от абсолютных значений.

In [ ]:
papb = [(0.01, 0.011), (0.01, 0.013), (0.10, 0.11), (0.10, 0.13)]

fig = go.Figure()

for pa, pb in papb:
    group_size = np.arange(100, 50000, 500)
    probs = [prob_pb_gt_pa(pa,  pb, n) for n in group_size]
    fig.add_trace(go.Scatter(x=group_size,
                             y=probs,
                             name=f'pa={pa:.1%}, pb={pb:.1%}'))
    
fig.update_layout(title='Probability B > A',
                  xaxis_title='Group Size',
                  yaxis_title='Prob')
fig.update_layout(yaxis_range=[0, 1])
fig.add_hline(y=0.5, line_dash="dash")
fig.add_hline(y=0.8, line_dash="dash")

fig.show()

Численно также можно оценить, сколько нужно измерений для достижения определенного уровня уверенности $P(p_B > p_A)$, например, 80% или 90%.  

Для конкретной пары $p_A$ и $p_B$ для этого можно построить изменение вероятности $P(p_B > p_A)$ с ростом $N$.  

Ниже - иллюстрация на основе средних значений в текущей выборке $p_A = s_A/N_A$, $p_B = s_B/N_B$:  

In [ ]:
pa = exp_info['p_mean']['A']
pb = exp_info['p_mean']['B']

group_size = np.arange(100, 100000, 500)
probs = np.array([prob_pb_gt_pa(pa, pb, n) for n in group_size])

fig = go.Figure()
fig.add_trace(go.Scatter(x=group_size, y=probs, name=f'pa={pa:.2%}, pb={pb:.2%}'))
fig.update_layout(title='Probability B > A',
                  xaxis_title='Group Size',
                  yaxis_title='Prob')
fig.update_layout(yaxis_range=[0, 1])
fig.add_hline(y=0.8, line_dash="dash")
fig.add_hline(y=0.95, line_dash="dash")
fig.show()


msgs = []
msgs.append(f"N_a = {exp_info['N']['A']}, N_b = {exp_info['N']['B']}")
msgs.append(f"Estimate pa = {pa:.2%}, pb = {pb:.2%}")
msgs.append(f"Current Prob(p_B > p_A) = {exp_info['Prob Optimal Variant']['B']}")
msgs.append(f"Estimate N to reach Prob(p_B > p_A) = 80%: {group_size[np.min(np.where(probs > 0.8))]}")
msgs.append(f"Estimate N to reach Prob(p_B > p_A) = 95%: {group_size[np.min(np.where(probs > 0.95))]}")

for m in msgs:
    print(m)

Для рассматриваемого примера текущая вероятность $P(p_B > p_A) = 78.7\%$ при $N=3000$ в каждой группе.  
Чтобы она достигла $95 \%$, нужно $N \sim 13000$ в каждой группе.

Для более точной оценки числа $N$ нужно проводить аналогичные вычисления с $p_A$ и $p_B$ из всех возможных значений апостериорного распределения.  
Т.е. сэмплируются $p_A$ и $p_B$, для каждой пары проводится оценка $N$ для достижения заданного уровня уверенности, строится распределение $N$.  
Для оценки длительности можно ориентироваться на среднее значение $E[N]$ в этом распределении.  

In [ ]:
n_sample = 100
post_sample_a = posterior_sample_for_binom_and_uniform_prior(exp_info['N_buy']['A'], exp_info['N']['A'], n_sample)
post_sample_b = posterior_sample_for_binom_and_uniform_prior(exp_info['N_buy']['B'], exp_info['N']['B'], n_sample)

N_95 = []

group_size = np.arange(100, 100000, 5000)
fig = go.Figure()
#todo: separate plotting from computation?

i = 0
for pa, pb in zip(post_sample_a, post_sample_b):
    probs = np.array([prob_pb_gt_pa(pa, pb, n) for n in group_size])
    Nreached = group_size[(probs > 0.95) | (probs < 0.05)]
    Nmin = np.min(Nreached) if Nreached.size > 0 else np.max(group_size)
    fig.add_trace(go.Scatter(x=group_size, y=probs, line_color='red', opacity=0.2, 
                             hovertemplate=f"pa={pa:.3f}, pb={pb:.3f}, N95={Nmin}"))
    N_95 = N_95 + [Nmin]
    i = i + 1
    if i % 10 == 0: print(f'finished {i}')

pa = exp_info['p_mean']['A']
pb = exp_info['p_mean']['B']
probs = np.array([prob_pb_gt_pa(pa, pb, n) for n in group_size])
Nreached = group_size[(probs > 0.95) | (probs < 0.05)]
Nmin = np.min(Nreached) if Nreached.size > 0 else np.max(group_size)
fig.add_trace(go.Scatter(x=group_size, y=probs, line_color='blue', opacity=0.6, 
                         hovertemplate=f"pa={pa:.3f}, pb={pb:.3f}, N95={Nmin}"))
N_95 = N_95 + [Nmin]
        
fig.update_layout(title='N to 95% certainty Scenarios',
                  xaxis_title='Group Size',
                  yaxis_title='Prob')
fig.update_layout(yaxis_range=[0, 1], showlegend=False)
fig.add_hline(y=0.95, line_dash="dash")
fig.add_hline(y=0.05, line_dash="dash")
fig.show()

fig = go.Figure()
fig.add_trace(go.Histogram(x=N_95, histnorm='probability', 
                           name='N to 95% certainty', marker_color='red',
                           opacity=0.6))
fig.update_layout(title='N to 95% certainty',
                  xaxis_title='N',
                  yaxis_title='% from total simulations',
                  barmode='overlay')
fig.show()

print(f'Expected N to 95% certainty: {np.mean(N_95)}')

На первом графике каждая линия соответствует конкретной паре $p_A$, $p_B$.  
Проводится симуляция эксперимента с такими значениями и строится верятность $P(p_B > p_A)$.   
Синяя линия - со средними значениями $p_A$ и $p_B$, как на предыдущем графике.


На гистрограмме - значение $N$ до достижения $P(p_B > p_A) = 95 \%$.  
В 60% случаев $N$ в пределах 20 тысяч.  
Ожидаемое значение $E[N] \sim 30k$. При оценке по средним $p_A$ и $p_B$ получалось $N \sim 13k$.  
Т.е. использование только одной пары $p_A$ и $p_B$ дает заниженную оценку.

По мере набора данных апостериорные распределения могут меняться.   
При необходимости можно сделать новую оценку, когда наберется больше данных. 

## Сколько должен продолжаться эксперимент?

По мере набора данных точность оценок параметров в каждой группе и уверенность в 
том, какая группа лучше, обычно повышается.  
До какого уровня уверенности нужно продолжать эксперимент - 60%, 80%, 90%?  

Можно целиться в достижение определенного занчения уверенности $P(p_B > p_A)$ - например, 95%.   
Но если $p_B > p_A$ с вероятностью 70%, есть ли смысл продолжать эксперимент дальше?  

Каждый день эксперимента - потери из-за невыбора лучшего варианта.  
Неверный выбор - потери после эксперимента.  
Поэтому чем более долгосрочными будут последствия, тем большая нужна уверенность в выборе.  

Также есть цена поддержки и цена реализации; можно считать, что цена поддержки одинакова,
цена реализации не учитывается.

Без эксперимента в среднем конвертировалось бы $p_A N$ пользователей.  
Если бы была выбрана группа B без эксперимента, то конвертировалось бы $p_B N$ пользователей.  
Если эксперимент запущен на 50% пользователей, то в среднем в день будет конвертироваться N (pa + pb)/2 пользователей.  
Когда будет сделан выбор в пользу одного из вариантов, количество пользователей изменится с N (pa + pb)/2 на $N p_A$ или $N p_B$ (на рисунке выбран вариант B).

На графике ниже - среднее количество успехов в группе B, среднее количество успехов в группе А и количество успехов в эксперименте. Трафик в эксперименте делится между двумя группами и в определенный момент принимается решение о выборе варианта B.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 20], y=[100, 100], 
                         line_dash="dash", line_color="red", name="Mean A"))
fig.add_trace(go.Scatter(x=[0, 20], y=[130, 130], 
                         line_dash="dash", line_color="blue", name="Mean B"))
fig.add_trace(go.Scatter(x=[0, 8, 8, 20], y=[115, 115, 130, 130], 
                         line_color="purple", name="exp"))
fig.add_vline(x=8, line_dash="dash", name="stop")
fig.update_layout(yaxis_range=[0, 200], xaxis_range=[0, 20])
fig.update_layout(xaxis_title='Days',
                  yaxis_title='Users',
                  hovermode="x")
fig.show()

Иногда деление трафика в эксперименте подстраивают пропорционально оценке "лучшести" каждой группы.
В этом случае число сконвертировавшихся пользователей в эксперименте со временем будет приближаться к лучшей группе (возможно после начальных колебаний). Однако вопрос "когда прекращать эксперимент" все равно остается.  

Нужно получить максимальное количество пользователей за время эксперимента и после выбора одного из вариантов.  

Если эксперимент еще не начался, то ожидаемое количество сконвертирующихся пользователей складывается из    
ожидаемого значения сконвертирующихся за время проведения эксперимента и ожидаемого количества тех, кто сконвертируется после выбора одного из вариантов 

$$
E[N_{s}] = E[N_{s-during-experiment}] + E[N_{s-after-choise}].
$$

Пусть суммарно в обеих группах в эксперименте примент участие $N_{exp}$ пользователей.  
$B_{split}$ - доля трафика в группе $B$.  
Тогда за время эксперимента

$$
E[N_{s-during-experiment}] = N_{exp} B_{split} p_B + N_{exp}(1 - B_{split}) p_A .
$$

Для оценки сконвертировавшихся пользователей после эксперимента нужно сделать оценку количества пользователей, которых затронет выбор того или иного варианта. Например, фича будет существовать еще год и повторных экспериментов или существенных изменений не будет. Т.е. последствия выбора того или иного варианта будут ощущаться все это время.  

Пусть за год всего будет $N$ пользователей, из которых $N_{exp}$ приняло участие в эксперименте.  
Тогда ожимдаемое число конверсий после прекращения эксперимента будет складыватсья из числа пользователей, не прошедших через эксперимент $N - N_{exp}$, умноженного на вероятность, что будет выбран вариант B и на конверсию в варианте B плюс аналогичное слагаемое для варианта A

$$
E[N_{s-after-choise}] = (N - N_{exp}) \big( p_{B} p_{B>A} + p_A (1 - p_{B>A}) \big).
$$

<После подстановки выражение для $E[N_{s}] = N p_A + N p_{B>A} \big( p_{B} - p_A \big) - N_{exp} (p_{B>A} - B_{split}) \big( p_{B} - p_A \big)$ >

In [ ]:
def expected_converted_users(pa, pb, p_b_gt_a, b_split_part, N_exp, N_total_affected):
    n_during_exp_s = (p_a * (1 - b_split_part) + p_b * b_split_part) * N_exp
    n_remaining_s = (N_total_affected - N_exp) * (p_b * p_b_gt_a + p_a * (1 - p_b_gt_a))
    N_s = n_during_exp_s + n_remaining_s
    return N_s

Ниже приведены оценки $E[N_s]$ и вероятности $P(p_B > p_A)$ с ростом $N_{exp}$ для нескольких значений $p_A$ и $p_B$.

In [ ]:
N_total_affected = 1000000
N_exp_vals = np.arange(0, N_total_affected, 10000)

p_a = 0.01
p_b = 1.05 * p_a
b_split_part = 0.5

fig = make_subplots(specs=[[{"secondary_y": True}]])
msgs = []

p_b_gt_a = [prob_pb_gt_pa(p_a, p_b, N_exp * (1 - b_split_part), N_exp * b_split_part) for N_exp in N_exp_vals]
N_s = [expected_converted_users(p_a, p_b, p_ba, b_split_part, N_exp, N_total_affected) for N_exp, p_ba in zip(N_exp_vals, p_b_gt_a)]

msgs.append(f'For pa={p_a:.2%}, pb={p_b:.2%} max expected N_s is for N_exp = {N_exp_vals[np.argmax(N_s)]} (N_s = {np.max(N_s)}, pb > pa: {p_b_gt_a[np.argmax(N_s)]})')
fig.add_trace(go.Scatter(x=N_exp_vals, y=N_s, mode='lines', line_color='red', name=f'pa={p_a:.2%}, pb={p_b:.2%}; Users'), secondary_y=False)
fig.add_trace(go.Scatter(x=N_exp_vals, y=p_b_gt_a, mode='lines', line_dash='dash', line_color='red', name=f'pa={p_a:.2%}, pb={p_b:.2%}; P(pb > pa)'), secondary_y=True)

p_a = 0.01
p_b = 1.1 * p_a
p_b_gt_a = [prob_pb_gt_pa(p_a, p_b, N_exp * (1 - b_split_part), N_exp * b_split_part) for N_exp in N_exp_vals]
N_s = [expected_converted_users(p_a, p_b, p_ba, b_split_part, N_exp, N_total_affected) for N_exp, p_ba in zip(N_exp_vals, p_b_gt_a)]

msgs.append(f'For pa={p_a:.2%}, pb={p_b:.2%} max expected N_s is for N_exp = {N_exp_vals[np.argmax(N_s)]} (N_s = {np.max(N_s)}, pb > pa: {p_b_gt_a[np.argmax(N_s)]})')
fig.add_trace(go.Scatter(x=N_exp_vals, y=N_s, mode='lines', line_color='blue', name=f'pa={p_a:.2%}, pb={p_b:.2%}; Users'), secondary_y=False)
fig.add_trace(go.Scatter(x=N_exp_vals, y=p_b_gt_a, mode='lines', line_dash='dash', line_color='blue', name=f'pa={p_a:.2%}, pb={p_b:.2%}; P(pb > pa)'), secondary_y=True)


fig.update_yaxes(title_text="Expected N_s", secondary_y=False)
fig.update_yaxes(title_text="P(B>A)", secondary_y=True)
fig.update_xaxes(title_text="Users in Experiment")
#fig.update_layout(yaxis_range=[10000, 11000], secondary_y=False)
fig.update_layout(hovermode='x')
fig.show()

for m in msgs:
    print(m)

Формально эксперимент стоит прекращать, когда достигнуто максимальное ожидаемое значение сконвертировавшихся пользователей.
Для $p_A=1.00\%, p_B=1.05\%$ максимальное ожидаемое количество конвертирующихся пользователей $max(E[N_s]) = 10398$ ожидается при $N_{exp} = 220000$. При этом $P(p_B > p_A) = 0.88$.

Фактически ожидаемое значение с какого-то момента может начать меняться слабо.  
Например, для $p_A=1\%$, $p_B=1.05\%$:  
$$
N_{exp} = 60k, \quad E[N_s] = 10357, \quad p(p_B > p_A) = 0.73;
\\
N_{exp} = 220k, \quad E[N_s] = 10398, \quad p(p_B > p_A) = 0.88;  
$$
Т.е. разница между ожидаемыми значениями $E[N_s]$ примерно 40 пользователей.  
Но при этом эксперимент можно прекратить более чем в 3 раза быстрее ($N_{exp} = 60k$ вместо $N_{exp} = 220k$).

Затягивание одного эксперимента может повышать цену поддержки а также мешать проведению других экспериментов.  
Это можно учесть, если при принятии решения сравнивать ценность сконвертировавшихся пользователей с ценой каждого дня эксперимента.
В таком случае вместо $E[N_{s}]$ решение будет приниматься по функции $E[Gain]$ вида

$$
E[Gain] = E[N_{s}] \cdot E[LTV] - Penalty(N_{exp}),
$$

где $E[LTV]$ - ценность сконвертировавшегося пользователя.

<Можно предположить, что $N \gg N_{exp}$. В этом случае ожидаемое значение сконвертировавшихся пользователей можно упростить до
$E[N_{s}] = N p_A + N p_{B>A} \big( p_{B} - p_A \big)$. $E[N_{s}]$ растет с ростом $p_{B>A}$. Однако рост замедляется и в какой-то момент дальнейший рост $E[N_{s}]$ не оправдывается издержками на поддержание эксперимента.>

Далее такие варианты использоваться не будут - будет использоваться только максимальное количество сконвертировавшихся пользователей.

Если эксперимент уже идет, то в ожидаемом значении нужно учесть фактически принявших участие пользователей $N_{fact}$ и фактически сконвертировавшихся $N_{s-fact}$.

$$
E[N_{s}] = N_{s-fact} + E[N_{s-during-remaining-experiment}] + E[N_{s-after-choise}] 
\\
E[N_{s-during-remaining-experiment}] = N_{further-exp} B_{split} p_B + N_{further-exp}(1 - B_{split}) p_A 
\\
E[N_{s-after-choise}] = (N - N_{fact} - N_{further-exp}) \big( p_{B} p_{B>A} + p_A (1 - p_{B>A}) \big) 
$$

Формально для принятия решения о прекращении:  
* По текущим данным построить апостериорные распределения $p_A$, $p_B$.
* Насэмплить несколько пар $p_A$, $p_B$.
* Для каждой пары построисть зависимость $E[N_{s}]$ от $N_{further-exp}$.
* Для каждой пары найти $N_{further-exp-max-s}$, соответствующее $N_{further-exp}$ при максимальном $E[N_{s}]$.
* Построить распределение $N_{further-exp-max-s}$.

Когда среднее по всем парам значение $N_{further-exp-max-s}$ будет меньше определенной величины - прекращать эксперимент.  

Ниже приведены оценки дополнительного числа пользователей в эксперименте для текущих данных и для случая, когда 
в эксперименте дополнительно поучаствовало 100000 тыс пользователей ("предельный" случай).

In [ ]:
#todo: If you have a procedure with 10 parameters ...

def expected_converted_users_fact(pa, pb, p_b_gt_a, b_split, N_s_fact, N_fact, N_further_exp, N_total_affected):
    n_s_during_exp = N_further_exp * (pa * (1 - b_split) + pb * b_split)
    n_s_after_choise = (N_total_affected - N_fact - N_further_exp) * (pb * p_b_gt_a + pa * (1 - p_b_gt_a))
    expected_n_s = N_s_fact + n_s_during_exp + n_s_after_choise
    return expected_n_s

def simulate_expected_ns(pa, pb, sa, sb, na, nb, b_split_part, N_total_affected):
    N_fact = na + nb
    N_s_fact = sa + sb
    N_further_exp_vals = np.arange(0, N_total_affected - N_fact, 10000)
    p_b_gt_a_sim = []
    N_s_sim = []
    for N_fe in N_further_exp_vals:
        na_fe = na + N_fe * (1 - b_split_part)
        nb_fe = nb + N_fe * b_split_part
        p_b_gt_a = prob_pb_gt_pa(pa, pb, na_fe, nb_fe)
        expected_n_s = expected_converted_users_fact(pa=pa, pb=pb, p_b_gt_a=p_b_gt_a,
                                                     b_split=b_split_part,
                                                     N_s_fact=N_s_fact, N_fact=N_fact,
                                                     N_further_exp=N_fe, N_total_affected=N_total_affected)
        p_b_gt_a_sim.append(p_b_gt_a)
        N_s_sim.append(expected_n_s)
    N_for_max_Ns = N_further_exp_vals[np.argmax(N_s_sim)]
    sim = {
        'pa': pa,
        'pb': pb,
        'N_exp': N_fact + N_further_exp_vals,
        'p_b_gt_a': np.array(p_b_gt_a_sim),
        'N_s': np.array(N_s_sim),
        'N_for_max_Ns': N_for_max_Ns
    }
    return sim

In [ ]:
pa_exact = exp_info['p_exact']['A']
pb_exact = exp_info['p_exact']['B']
N_total_affected = 1000000
b_split_part = 0.5
n_sample = 50

####### current data

sims_current = []

na = exp_info['N']['A']
nb = exp_info['N']['B']
sa = exp_info['N_buy']['A']
sb = exp_info['N_buy']['B']
    
post_sample_a = posterior_sample_for_binom_and_uniform_prior(sa, na, n_sample)
post_sample_b = posterior_sample_for_binom_and_uniform_prior(sb, nb, n_sample)
    
for i, (pa, pb) in enumerate(zip(post_sample_a, post_sample_b)):
    sim = simulate_expected_ns(pa, pb, sa, sb, na, nb, b_split_part, N_total_affected)
    sims_current.append(sim)
    if (i + 1) % 10 == 0: print(f'finished {i + 1} simulations for current data')

#print(sims_current[-1])

####### additional data

sims_additional = []

additional_data = 100000
na = exp_info['N']['A'] + int(additional_data * (1 - b_split_part))
nb = exp_info['N']['B'] + int(additional_data * b_split_part)
sa = exp_info['N_buy']['A'] + np.random.binomial(na, pa_exact)
sb = exp_info['N_buy']['B'] + np.random.binomial(nb, pb_exact)
    
post_sample_a = posterior_sample_for_binom_and_uniform_prior(sa, na, n_sample)
post_sample_b = posterior_sample_for_binom_and_uniform_prior(sb, nb, n_sample)
    
for i, (pa, pb) in enumerate(zip(post_sample_a, post_sample_b)):
    sim = simulate_expected_ns(pa, pb, sa, sb, na, nb, b_split_part, N_total_affected)
    sims_additional.append(sim)
    if (i + 1) % 10 == 0: print(f'finished {i + 1} simulations for additional data')
        
#print(sims_additional[-1])

In [ ]:
fig = make_subplots(rows=2, cols=2)

N_for_max_Ns_current = []
for sim in sims_current:
    fig.add_trace(go.Scatter(x=sim['N_exp'], y=sim['N_s'], line_color='red', opacity=0.2,
                             hovertemplate="N_exp=%{x}, E[Ns]=%{y} <br>" + f"pa={sim['pa']:.3f}, pb={sim['pb']:.3f}, N_for_max_Ns={sim['N_for_max_Ns']}"),
                  row=1, col=1)
    N_for_max_Ns_current.append(sim['N_for_max_Ns'])
counts, edges = np.histogram(N_for_max_Ns_current, bins=20, range=(0, N_total_affected))
fig.add_trace(go.Bar(x=edges, y=counts / np.sum(counts),
                     name='N to max Ns', 
                     opacity=0.6, marker_color='red'),
              row=2, col=1)

N_for_max_Ns_additional_data = []
for sim in sims_additional:
    fig.add_trace(go.Scatter(x=sim['N_exp'], y=sim['N_s'], line_color='red', opacity=0.2,
                             hovertemplate="N_exp=%{x}, E[Ns]=%{y} <br>" + f"pa={sim['pa']:.3f}, pb={sim['pb']:.3f}, N_for_max_Ns={sim['N_for_max_Ns']}"),
                  row=1, col=2)
    N_for_max_Ns_additional_data.append(sim['N_for_max_Ns'])
counts, edges = np.histogram(N_for_max_Ns_additional_data, bins=20, range=(0, N_total_affected))
fig.add_trace(go.Bar(x=edges, y=counts / np.sum(counts),
                     name='N to max Ns', 
                     opacity=0.6, marker_color='red'),
              row=2, col=2)

fig.update_layout(autosize=False, 
                  width=1000,
                  height=700)
fig.update_layout(title='Experiment Duration')
fig.update_layout(showlegend=False)
fig['layout']['xaxis'].update(title_text='N exp', range=[0, N_total_affected])
fig['layout']['xaxis2'].update(title_text='N exp', range=[-20000, N_total_affected])
fig['layout']['xaxis3'].update(title_text='Additional N to max E[Ns]', range=[-20000, N_total_affected])
fig['layout']['xaxis4'].update(title_text='Additional N to max E[Ns]', range=[-20000, N_total_affected])
fig['layout']['yaxis'].update(title_text='E[Ns]')
fig['layout']['yaxis2'].update(title_text='E[Ns]')
fig['layout']['yaxis3'].update(title_text='Part of Simulations')
fig['layout']['yaxis4'].update(title_text='Part of Simulations')
fig.show()

print(f'Expected N to max Ns for current data: {np.mean(N_for_max_Ns_current)}')
print(f'Expected N to max Ns with additional data: {np.mean(N_for_max_Ns_additional_data)}')

При достаточно большом количестве пользователей в эксперименте во всех сценариях оценка дальнейшего числа конверсий только уменьшается. Поэтому проводить эксперимент дальше не целесообразно.

При таком подходе все еще остается некоторая степень субъективности при выборе момента прекращения эксперимента.  
Ожидаемое значение $N_{further-exp}$ до максимального $E[N_s]$ не всегда будет равно 0.  
Поэтому нужно выбрать произвольное значение ниже которого продолжать эксперимент нет смысла.  
Выбор становится более объективным, если есть цена каждого дня эксперимента.

На практике эффекты меньше определенной величины - например, меньше 5% - могут быть не интересны.  
Для предварительной оценки длительности эксперимента можно выбрать $p_A$ равным историческому значению конверсии и $p_B = 1.05 p_A$.    
В последствии длительность можно будет уточнить по ходу проведения эксперимента. 

### TODO: Количество правильно принятых решений

Генерировать pa, pb;  
pa - [1% -- 20%]  
pb - [0.8 -- 1.2] * pa  

pa известно; pb - нет.  

пытаться угадать, какой вариант лучше, за минимальное количество N_exp.

In [ ]:
n_guess = 1000

pa_guess = np.random.uniform(low=0.01, high=0.2, size=n_guess)
ba = np.random.uniform(low=0.8, high=1.2, size=n_guess)
pb_guess = ba * pa
pa[:5], pb[:5], ba[:5]

In [ ]:
def estimate_mean_further_n(sa, na, sb, nb, n_sample):
    sims = []
    N_for_max_Ns = []
    N_total_affected = 1000000
    b_split_part = nb / (na + nb)
    post_sample_a = posterior_sample_for_binom_and_uniform_prior(sa, na, n_sample)
    post_sample_b = posterior_sample_for_binom_and_uniform_prior(sb, nb, n_sample)
    for i, (pa, pb) in enumerate(zip(post_sample_a, post_sample_b)):
        sim = simulate_expected_ns(pa, pb, sa, sb, na, nb, b_split_part, N_total_affected)
        sims.append(sim)
        #if (i + 1) % 10 == 0: print(f'finished {i + 1} simulations for {i, pa_exact, pb_exact}')
        N_for_max_Ns.append(sim['N_for_max_Ns'])
    return np.mean(N_for_max_Ns_current)

In [ ]:
#for pa, pb in pa_guess, pb_guess:
pa_exact = pa[0]
pb_exact = pb[0]

expected_n_for_max_nx = estimate_duration_for_papb(pa=pa_exact, pb=1.05*pa_exact)
print(f"Initial duration estimate: {expected_n_for_max_nx}")

na_daily = 5000
nb_daily = 5000

na_total = []
nb_total = []

na_s = []
nb_s = []

pa_est = []
pb_est

expected_n_for_max_ns = []

#pandas:
#pa_exact, pb_exact, iteration, na, na_s, nb_nb_s, pa, pb, pb/pa, pb>pa, further_n
i = 0

while expected_n_for_max_ns_i > 0:
    i = i + 1
    na_total = na_total + [na_daily]
    nb_total = nb_total + [nb_daily]
    na_s = na_s + [np.random.binomial(na_daily, pa_exact)]
    nb_s = nb_s + [np.random.binomial(nb_daily, pb_exact)]
    pa_est = pa_est + [np.sum(na_s) / np.sum(na_total)]
    pb_est = pb_est + [np.sum(nb_s) / np.sum(nb_total)]
    pb_gt_pa = prob_pb_gt_pa(na_accum_last, nas_accum_last, nb_accum_last, nbs_accum_last)
    n_sample = 30
    expected_n_for_max_ns_i = estimate_mean_further_nexp(nas_accum_last, na_accum_last, 
                                                         nbs_accum_last, nb_accum_last, 
                                                         n_sample)
    expected_n_for_max_ns = expected_n_for_max_ns + [expected_n_for_max_ns_i]
    print('i')

In [ ]:
print('#guesses {}, #correct guesses: {}')
print('Total Nexp: ')

## TODO: Проверка модели

## Дополнение: динамика по дням

Тест часто идет несколько дней.  
Удобно видеть динамику по дням.

Пусть параметры теста следующие:

In [ ]:
exp_info = pd.DataFrame([
    {'group' :'A', 'p_exact': 0.02, 'n_days': 30, 'N_daily': 1000},
    {'group': 'B', 'p_exact': 0.022, 'n_days': 30, 'N_daily': 1000}],
    columns=['group', 'p_exact', 'n_days', 'N_daily']
).set_index('group')
exp_info

Данные

In [ ]:
results_a = np.random.binomial(n=exp_info['N_daily']['A'], p=exp_info['p_exact']['A'], size=exp_info['n_days']['A'])
results_b = np.random.binomial(n=exp_info['N_daily']['B'], p=exp_info['p_exact']['B'], size=exp_info['n_days']['B'])

df = pd.concat([
    pd.DataFrame([('A', d, exp_info['N_daily']['A'], s) for d, s in enumerate(results_a)], 
                 columns=['group', 'day', 'n_users', 'buys']),
    pd.DataFrame([('B', d, exp_info['N_daily']['B'], s) for d, s in enumerate(results_b)], 
                 columns=['group', 'day', 'n_users', 'buys'])
])

display(df.head())
display(df.tail())

Конверсии по дням

In [ ]:
df['p_daily'] = df['buys'] / df['n_users']
display(df.head())

fig = px.line(df, x='day', y='p_daily', color='group', markers=True)
fig.show()

Конверсии по накопленным данным

In [ ]:
df_accum = df.groupby('group')['n_users', 'buys'].cumsum().rename(columns={'n_users': 'n_users_accum', 'buys':'buys_accum'})
display(df_accum.head())

df = pd.concat([df, df_accum], axis=1)
df['p_accum'] = df['buys_accum'] / df['n_users_accum']
display(df.head())

fig = px.line(df, x='day', y='p_accum', color='group', markers=True)
fig.show()

HPDI:

In [ ]:
hpdi = 0.9
df[['p_hpdi_lower','p_hpdi_higher']] = df.apply(lambda row: pd.Series(hpdi_for_binom_and_uniform_prior(hpdi, row['buys_accum'], row['n_users_accum'])), axis=1)
df['error_lower'] = df['p_accum'] - df['p_hpdi_lower']
df['error_higher'] = df['p_hpdi_higher'] - df['p_accum']
display(df.head())


fig = go.Figure()
fig.add_trace(go.Scatter(x=df[df['group'] == 'A']['day'], 
                         y=df[df['group'] == 'A']['p_accum'],
                         mode='lines+markers', name='A', line_color='blue'))
fig.add_trace(go.Scatter(x=pd.concat([df[df['group'] == 'A']['day'], df[df['group'] == 'A']['day'][::-1], df[df['group'] == 'A']['day'][0:1]]), 
                         y=pd.concat([df[df['group'] == 'A']['p_hpdi_higher'], df[df['group'] == 'A']['p_hpdi_lower'][::-1], df[df['group'] == 'A']['p_hpdi_higher'][0:1]]),
                         fill='toself', name=f'{hpdi:.0%} HPDI A',
                         hoveron = 'points+fills',
                         hoverinfo = 'text+x+y',
                         line_color='blue', fillcolor='blue', opacity=0.4))
fig.add_trace(go.Scatter(x=df[df['group'] == 'B']['day'], 
                         y=df[df['group'] == 'B']['p_accum'],
                         mode='lines+markers', name='B', line_color='red'))
fig.add_trace(go.Scatter(x=pd.concat([df[df['group'] == 'B']['day'], df[df['group'] == 'B']['day'][::-1], df[df['group'] == 'B']['day'][0:1]]), 
                         y=pd.concat([df[df['group'] == 'B']['p_hpdi_higher'], df[df['group'] == 'B']['p_hpdi_lower'][::-1], df[df['group'] == 'B']['p_hpdi_higher'][0:1]]),
                         fill='toself', name=f'{hpdi:.0%} HPDI B',
                         hoveron = 'points+fills',
                         hoverinfo = 'text+x+y',
                         line_color='red', fillcolor='red', opacity=0.4))
fig.update_layout(xaxis_title='Days',
                  yaxis_title='P',
                  hovermode="x")
fig.update_layout(height=470)
fig.show()

In [ ]:
# pd.concat([df[df['group'] == 'A']['day'],
#            df[df['group'] == 'A']['day'][::-1],
#            df[df['group'] == 'A']['day'][0:1]])
# pd.concat([df[df['group'] == 'A']['p_hpdi_higher'],
#            df[df['group'] == 'A']['p_hpdi_lower'][::-1],
#            df[df['group'] == 'A']['p_hpdi_higher'][0:1]])

In [ ]:
#prob_pb_ge_pa_for_binom_and_uniform_prior(df[])

Вероятность $P(p_B > p_A)$

In [ ]:
widedf = df.set_index(['group', 'day']).unstack(level=0)
#display(widedf.head())

widedf['pb_gt_pa'] = widedf.apply(lambda row: prob_pb_gt_pa(
    p_a=row['p_accum']['A'],
    p_b=row['p_accum']['B'],
    N_a=row['n_users_accum']['A'], 
    N_b=row['n_users_accum']['B']), axis=1)
widedf = widedf.reset_index()
#display(widedf.head())

fig = px.line(widedf, x='day', y='pb_gt_pa', markers=True, title="$P_b >= P_a$",
             labels={"day": "Day", "pb_gt_pa": ""})
fig.add_hline(y=0.5, line_dash="dash")
fig.update_layout(
    yaxis_range=[0, 1],
    yaxis_tickformat = ',.0%')
fig.show()

#TODO: ожидаемое количество сконвертировавшихся, прогноз pb>pa. 

# Приложение: сопряженное априорное распределение к биномиальному

Можно показать, что если функция правдоподобия задана биномиальным распределением, априорная вероятность - бета-распределением, то апостериорная вероятность также будет выражаться бета-распределением, но с другими значениями параметров. Говорят, что бета-распределение является сопряженным априорным распределением к биномиальному [[ConjPrior](https://en.wikipedia.org/wiki/Conjugate_prior)]. Это позволяет упростить расчеты в некоторых случаях.

$$
P(p | data) = \frac{ P(data | p) P(p) }{P(data)}
\propto
P(data | p) P(p)
$$

$$
P(data | p) = Binom(p, s, N) \propto p^s (1-p)^{N-s}
$$

$$
P(p) = Beta(p; \alpha, \beta) \propto p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
P(p | data) 
\propto P(data | p) P(p) = Binom(p, s, N) Beta(p; \alpha, \beta) 
\propto p^{s + \alpha - 1} (1-p)^{N - s + \beta - 1}
$$

$$
P(p | data) = Beta(p; \alpha', \beta')
\\
\alpha' = \alpha + s, \qquad \beta' = \beta + (N-s)
$$

Вид бета-распределения при различных значениях параметров приведен ниже.   
При $\alpha = 1, \beta=1$ бета-распределение совпадает с равномерным.  
По мере набора данных можно ожидать роста $s$ и $N$ при постоянном отношении $s/N \approx p_{exact}$.  
Бета-распределение будет локализовываться вокруг этого отношения.  

In [ ]:
x = np.linspace(0, 1, 1000)

fig = go.Figure()
for a, b in [(1, 1), (2, 10), (10, 50), (20, 100)]:
    fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a, b), mode='lines', name=f'a={a}, b={b}'))
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob',
                  hovermode="x")
fig.show()

Можно задавать априорное распределение локализованным вокруг определенного значения вместо равномерного распределения.

## Заключение

Основные шаги оценки эксперимента в байесовском подходе:

* выбрать модель
* по историческим данным проверить, что модель в состоянии описать данные
* для оценки длительности эксперимента задаться априорными распределениями параметров в каждой группе;
промоделировать различные сценарии; оценить ожидаемую длительность до наибольшей "выгоды" за время эксперимента.  
Это предварительноая оценка - фиксировать это значение не обязательно.
* по мере проведения эксперимента делать оценки параметров для экспериментальных групп и обновлять оценки длительности
* прекратить эксперимент, когда когда ожидаемое значение "выгоды" начнет снижаться

Что не обсуждалось:

* Оценка метрик, не являющихся конверсиями. Например, выручки на пользователя или среднего чека.    

* Выбор более информативного априорного распределения, чем равномерное.
Можно выбрать распределение с учетом исторического значения конверсии. Это позволит уменьшить размер выборки, требуемый для достижения значимого отличия.

* Более сложные модели. Пользователей часто удобно объединить в сегменты по похожему поведению в сервисе или по различным рекламным каналам. Поэтому вместо биномиального распределения может быть удобнее использовать смесь биномиальных распределений с различными параметрами для каждого сегмента. 

* Модели с большим числом параметров; генерация сэмплов апостериорного распределения методами Монте-Карло с марковскими цепями.

* Сравнение байесовского подхода с альтернативными методами оценки экспериментов. В том числе с методом проверки статистических гипотез.

## Благодарности


## Ссылки
[MicroExp] R. Kohavi et al, [Online Experimentation at Microsoft.](https://www.microsoft.com/en-us/research/publication/online-experimentation-at-microsoft/)  
[TrustworthyAB] R. Kohavi, D. Tang, Y. Xu, [Trustworthy Online Controlled Experiments: A Practical Guide to A/B Testing.](https://www.amazon.com/gp/product/B0845Y3DJV/ref=dbs_a_def_rwt_hsch_vapi_tkin_p1_i0)  
[SGBS] B. Lambert, A Student’s Guide to Bayesian Statistics ( [Textbook](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [Student Resources](https://study.sagepub.com/lambert) ).   
[SR] R. McElreath, Statistical Rethinking: A Bayesian Course with Examples in R and STAN ( [Textbook](https://www.amazon.co.uk/Statistical-Rethinking-Bayesian-Examples-Chapman/dp/036713991X/ref=sr_1_1), [Video Lectures](https://www.youtube.com/playlist?list=PLDcUM9US4XdMROZ57-OIRtIK0aOynbgZN), [Course Materials](https://github.com/rmcelreath/stat_rethinking_2022) ).     
[SubjProb] [Subjectivism](https://en.wikipedia.org/wiki/Probability_interpretations#Subjectivism), in *Probability Interpretations*, *Wikipedia.*   
[UU] D.V. Lindley, [Understanding Uncertainty.](https://www.amazon.co.uk/Understanding-Uncertainty-Wiley-Probability-Statistics-ebook/dp/B00GYVM33Q)      
[BernoulliProcess] [Bernoulli Process](https://en.wikipedia.org/wiki/Bernoulli_process), *Wikipedia.*     
[BinomDist] [Binomial Distribution](https://en.wikipedia.org/wiki/Binomial_distribution), *Wikipedia.*  
[SciPyBinom] [scipy.stats.binom](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html), *SciPy Reference.*   
[BetaDist] [Beta Distribution](https://en.wikipedia.org/wiki/Beta_distribution), *Wikipedia.*     
[SciPyBeta] [scipy.stats.beta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html), *SciPy Reference.*    
[ConjPrior] [Conjugate Prior](https://en.wikipedia.org/wiki/Conjugate_prior), *Wikipedia.*   
[ProbConv] [Convolution of Probability Distributions](https://en.wikipedia.org/wiki/Convolution_of_probability_distributions), *Wikipedia.*   
[ProbRatio] [Ratio Distribution](https://en.wikipedia.org/wiki/Ratio_distribution), *Wikipedia.*  
[CauchyDist] [Cauchy Distribution](https://en.wikipedia.org/wiki/Cauchy_distribution), *Wikipedia.*  